# Setup 

In [1]:
!pip install transformers torch datasets pip accelerate
!pip install transformers --upgrade
!pip install ipywidgets
!pip install python-dotenv pickleshare

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.9.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.10.23-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata 

# Training small pre-trained ByT5 model using dataset

In [4]:
# from dotenv import load_dotenv
# import os

# load_dotenv()

# HF_TOKEN = os.getenv("HF_TOKEN")

# # print("HF_TOKEN: ", HF_TOKEN)

# # source: https://huggingface.co/docs/transformers/v4.28.1/tasks/summarization
# from huggingface_hub import login

# login(HF_TOKEN)
LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","mul","deu_eng"]
LANGS = LANGS[-2:]
LANGS = ["mul_2","amh","pol","ote"]
LANGS = ["mul_2","amh","pol","ote"]
LANGS = ["mul-language-code","mul-language-word"]
LANGS = ["mul-language-code"]
LANGS = ["5lang-sample"]


PATH_JSON_DATA = "../preprocessing/preprocessing_to_json/data/"

for lang in LANGS:
    from datasets import load_dataset
    # dataset= dataset.train_test_split(test_size=0.2) #train_size=10000,
    dataset = load_dataset("json", data_files={"train": PATH_JSON_DATA + lang + "_trn.json", "validation": PATH_JSON_DATA + lang + "_dev.json"})

    print(dataset["train"][0])

    from transformers import AutoTokenizer

    checkpoint = "google/byt5-small"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    # -

    import torch

    from transformers import (
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        AutoModelForSeq2SeqLM,
        Seq2SeqTrainingArguments,
        Seq2SeqTrainer
    )
    from datasets import load_dataset

    def preprocess_function(examples):
        # inputs = [prefix + doc for doc in examples["text"]]
        inputs = examples["input"]
        model_inputs = tokenizer(inputs)

        labels = tokenizer(text_target=examples["target"]) # same length irrelevant here, so no truncation or padding

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs


    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    print(tokenized_dataset)
    sample = tokenized_dataset["train"][0]
    print(sample["labels"])

    from transformers import DataCollatorForSeq2Seq

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

    # import evaluate

    # rouge = evaluate.load("rouge")

    # import numpy as np


    # def compute_metrics(eval_pred):
    #     predictions, labels = eval_pred
    #     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    #     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    #     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    #     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    #     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    #     result["gen_len"] = np.mean(prediction_lens)

    #     return {k: round(v, 4) for k, v in result.items()}

    # import numpy as np
    # import evaluate

    # metric = evaluate.load("accuracy")

    # def compute_metrics(eval_pred):
    #     predictions, labels = eval_pred # TODO: labels vs title?

    #     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    #     labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # TODO: correct?

    #     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #     return metric.compute(predictions=predictions, references=labels)

    from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    training_args = Seq2SeqTrainingArguments(
        output_dir="byt5_small/"+lang,
        eval_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=4,
        predict_with_generate=True,
        # save_strategy="epoch",
        fp16=False,# did not work with True #change to bf16=True for XPU 
        #bf16 = True, # performance was best without setting fp16 or bf16 to true
        push_to_hub=True,
        warmup_steps = 500,
        # load_best_model_at_end=True, # otherwise not model with minimum loss during training, like https://huggingface.co/docs/transformers/tasks/sequence_classification
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.push_to_hub()


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'input': 'Navajo: Inflect náʼáłkad | V;IND;PFV;NOM(1,GRPL)', 'target': 'ńdaʼiilkad'}


2025-11-01 16:16:20.882874: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})
[200, 135, 103, 100, 205, 191, 108, 108, 111, 110, 100, 103, 1]


/tmp/ipykernel_301522/2453831306.py:129: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,2.696000,0.715448
2,0.788000,0.518034
3,0.591400,0.467004
4,0.488600,0.454720


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

# Testing models

In [8]:
# SETTINGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","mul","deu_eng"] + ["mul_2","ote","pol","amh"] + ["mul-language-code","mul-language-word"]
SETTINGS = ["mul"]
SETTINGS = ["5lang-sample"]
TEST_LANGS_OLD = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"] 
TEST_LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"] + ["ote","pol","amh","csb"]
lang5_LANGS = ["deu","nav","klr","amh","grc"]
lang_code = ""
for lang_setting in SETTINGS:
    if lang_setting == "mul": LANG_CODES = ["-language-word","-language-code",""] # test with no language code/word too
    else: LANG_CODES = [""]
    for lang_code in LANG_CODES:
        if lang_setting in ( "deu_eng", "mul") and lang_code == "": test_langs = TEST_LANGS_OLD
        elif lang_setting in ["mul_2","mul"]: test_langs = TEST_LANGS # test multilingual model on all languages it was trained on + Kashubian
        elif lang_setting == "5lang-sample": test_langs = lang5_LANGS
        elif lang_setting == "pol": test_langs = ["pol","csb"] # test Polish model on Kashubian also
        else: test_langs = [lang_setting]
        for test_lang in test_langs:
            import torch
            from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
            PRETRAINED_MODEL = "google/byt5-small"
            FINETUNED_MODEL ="livles/"+lang_setting + lang_code
            #FINETUNED_MODEL ="./byt5_small/"+lang_setting + "-" + lang_code
            MODEL = FINETUNED_MODEL
            print(MODEL)
            tokenizer = AutoTokenizer.from_pretrained(
                FINETUNED_MODEL,
            )
            model = AutoModelForSeq2SeqLM.from_pretrained(
                FINETUNED_MODEL,
            dtype=torch.float16,
            device_map="auto"
            )

            from datasets import load_dataset
            OUT_DIR = "output_byt5_small/" + lang_setting + lang_code+ "/" 
            PATH_TO_JSON_TEST_FILE = "../preprocessing/preprocessing_to_json/data/" + test_lang + lang_code+ "_test.json"
            PATH_TO_ORIG_TEST_FILE = "../2023InflectionST/part1/data/" + test_lang + ".tst"
            with open (OUT_DIR + test_lang + lang_code + "_byt5-small.out","w") as out_file, open (PATH_TO_ORIG_TEST_FILE, "r") as test_file:
                COUNTER = 0
                dataset = load_dataset("json",data_files={"test":PATH_TO_JSON_TEST_FILE})
                covered_test_lines = dataset["test"]["input"]
                LEN = len(covered_test_lines)


                # preds = map (generate,covered_test_lines)
                for line in covered_test_lines:
                    input_ids = tokenizer(line, return_tensors="pt").to(model.device)
                    output = model.generate(
                        **input_ids,
                        max_new_tokens=50,
                        num_beams=4,
                        #early_stopping=True
                    )
                    output_string = (tokenizer.decode(output[0], skip_special_tokens=True))
                    lemma, features, ref = test_file.readline().strip().split("\t")

                    # evaluate
                    if (ref == output_string):
                        COUNTER += 1
                        print(ref,output_string)
                    else:
                        print("ref:",ref,"pred:",output_string)
                    
                    # write
                    out_file.write(lemma + "\t" +features+ "\t" + output_string + "\n")
                    out_file.flush()

                accuracy = COUNTER / LEN
                out_file.write("accuracy:"+ str(accuracy))
                print(accuracy)

livles/5lang-sample


Generating test split: 0 examples [00:00, ? examples/s]

ref: Aikido pred: Aikidookido
ref: Aikido pred: Aikidookido
ref: Aikidos pred: Aikidookido
ref: Aikido pred: Aikidookido
Anreisen Anreisen
Anreise Anreise
Anreisen Anreisen
Anreise Anreise
Anreisen Anreisen
ref: Anreise pred: Anreisest
Anreisen Anreisen
Anreise Anreise
ref: Arsenikesser pred: Arsenikessern
Arsenikesser Arsenikesser
Arsenikessern Arsenikessern
Arsenikesser Arsenikesser
ref: Arsenikesser pred: Arsenikessern
ref: Arsenikessers pred: Arsenikesser
ref: Arsenikesser pred: Arsenikessern
Arsenikesser Arsenikesser
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
ref: Auslassungszeichens pred: Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Bergenten Bergenten
Bergente Bergente
Bergenten Bergenten
Bergente Bergente
Bergenten Bergenten
Bergente Bergente
Bergenten Bergenten
Bergente Bergente
ref:

Generating test split: 0 examples [00:00, ? examples/s]

ref: daʼdííníildíín pred: adiníłdíín
ref: adííníildíín pred: adiníłdíín
ref: adííníshdíín pred: adiníłdíín
ref: daʼdíínółdíín pred: adiníłdíín
ref: adíínółdíín pred: adiníłdíín
ref: adííníłdíín pred: adiníłdíín
ref: daʼdiníłdíín pred: adiníłdíín
adiníłdíín adiníłdíín
adiníłdíín adiníłdíín
ref: dazhdiʼníłdíín pred: jidiníłdíín
ref: adizhníłdíín pred: jííníłdíín
ref: adizhníłdíín pred: jidiníłdíín
ref: ahidaniiltʼé pred: aheełtʼé
ref: ahiniiltʼé pred: aheełtʼé
ref: ahinishtʼé pred: aheełtʼé
ref: ahidanołtʼé pred: aheełtʼé
ref: ahinołtʼé pred: aheełtʼé
ref: ahiníłtʼé pred: aheełtʼé
ref: ahidaałtʼé pred: aheełtʼé
aheełtʼé aheełtʼé
aheełtʼé aheełtʼé
ref: ahidajíłtʼé pred: jííłtʼé
ref: ahijíłtʼé pred: jííłtʼé
ref: ahijíłtʼé pred: jííłtʼé
ref: dadiniilyił pred: diłhił
ref: diniilyił pred: diłhił
ref: dinishhił pred: diłhił
ref: dadinołhił pred: diłhił
ref: dinołhił pred: diłhił
ref: diníłhił pred: diłhił
ref: dadiłhił pred: diłhił
diłhił diłhił
diłhił diłhił
ref: dazhdiłhił pred: diłhił
ref: 

...

abalienieren abalienieren
abalienierten abalienierten
abalienierte abalienierte
abalieniertest abalieniertest
abalienierten abalienierten
abalienierte abalienierte
ref: abgelaufen pred: gelauft
lauf ab lauf ab
laufen ab laufen ab
laufe ab laufe ab
ref: läuft ab pred: lauft ab
ref: liefen ab pred: lauften ab
ref: liefst ab pred: lauftest ab
ref: liefen ab pred: lauften ab
ref: lief ab pred: laufte ab
ref: liefen ab pred: lauften ab
ref: liefe ab pred: laufte ab
ref: liefest ab pred: laufest ab
applicirend applicirend
ref: applicirt pred: geapplicirt
applicirt applicirt
appliciren appliciren
applicirtet applicirtet
applicirten applicirten
applicirte applicirte
appliciret appliciret
applicirest applicirest
appliciren appliciren
applicirten applicirten
applicirte applicirte
applicirte applicirte
heb auf heb auf
hebt auf hebt auf
ref: hob auf pred: habte auf
ref: hobt auf pred: habtet auf
ref: hoben auf pred: haben auf
ref: hob auf pred: habte auf
hebet auf hebet auf
hebest auf hebest auf
h

gemulcht gemulcht
mulch mulch
mulchen mulchen
mulchen mulchen
mulcht mulcht
mulchten mulchten
mulchte mulchte
mulchten mulchten
mulchet mulchet
mulchten mulchten
mulchte mulchte
mulchten mulchten
genützt genützt
nützten nützten
nützte nützte
nützen nützen
nütze nütze
nützet nützet
nützest nützest
nützen nützen
nütze nütze
nützte nützte
nütztest nütztest
nützte nützte
paraphrasierend paraphrasierend
paraphrasiert paraphrasiert
paraphrasiert paraphrasiert
paraphrasierten paraphrasierten
paraphrasiertet paraphrasiertet
paraphrasieren paraphrasieren
paraphrasiere paraphrasiere
paraphrasierten paraphrasierten
paraphrasierte paraphrasierte
paraphrasiertet paraphrasiertet
paraphrasiertest paraphrasiertest
paraphrasierten paraphrasierten
paraphrasierte paraphrasierte
pubertiert pubertiert
pubertier pubertier
pubertieren pubertieren
pubertiert pubertiert
pubertiertet pubertiertet
pubertiertest pubertiertest
pubertieren pubertieren
pubertieret pubertieret
pubertieren pubertieren
pubertiere puber

ref: adeesǫ́ǫ́s pred: adeeshzǫ́ǫ́z
ref: adadoohsǫ́ǫ́s pred: adadoohsǫ́ǫ́z
ref: adeidoozǫ́ǫ́s pred: adadoozǫ́ǫ́z
ref: iidoozǫ́ǫ́s pred: adoozǫ́ǫ́z
ref: adazhdoozǫ́ǫ́s pred: adazhdoozǫ́ǫ́z
ref: azhdoozǫ́ǫ́s pred: azhdoozǫ́ǫ́z
ref: azhdoozǫ́ǫ́s pred: azhdoozǫ́ǫ́z
ref: ndeiidá pred: nideiidá
ref: ndaahdá pred: nidaahdá
ref: nanidá pred: ninidá
ref: ndaadá pred: nidaadá
ref: njidá pred: nijidá
ref: nisiidzá pred: nisiidaa
ref: ndasoohdzá pred: nidasoodaa
ref: nisoohdzá pred: nisoodaa
ref: nisínídzá pred: nisínídą́ą́ʼ
ref: ndaasdzá pred: nidaadá
ref: naasdzá pred: naasdá
ref: naasdzá pred: niyíídą́ą́ʼ
ref: ndajisdzá pred: nidajizdą́
ref: njisdzá pred: nijizdą́
ref: njisdzá pred: nijizdą́
ndadiidaał ndadiidaał
ref: ndiidaał pred: ndiildaał
ndadoohdaał ndadoohdaał
ref: ndíídaał pred: ndíídoodaał
ndadoodaał ndadoodaał
ref: ndoodaał pred: nidoodaał
ndazhdoodaał ndazhdoodaał
nizhdoodaał nizhdoodaał
ref: ndeiiljooł pred: nideiiljooł
ref: neiiljooł pred: niiljooł
naashjooł naashjooł
ref: ndaałjooł 

diichʼił diichʼił
doohchʼił doohchʼił
dííchʼił dííchʼił
ref: deidoochʼił pred: dadoochʼił
ref: yidoochʼił pred: doochʼił
jidoochʼił jidoochʼił
ref: dadiitsʼį́į́h pred: deiitsʼį́į́h
diitsʼį́į́h diitsʼį́į́h
ref: diistsʼį́į́h pred: deistsʼį́į́h
ref: doohtsʼį́į́h pred: wohtsʼį́į́h
ref: diitsʼį́į́h pred: nidiitsʼį́į́h
ref: yidiitsʼį́į́h pred: diitsʼį́į́h
ref: dazhdiitsʼį́į́h pred: dajidiitsʼį́į́h
ref: dadisiitsʼą́ą́ʼ pred: dadiitsʼį́į́ʼ
ref: disiitsʼą́ą́ʼ pred: diitsʼį́į́ʼ
ref: disétsʼą́ą́ʼ pred: díítsʼį́į́ʼ
ref: disootsʼą́ą́ʼ pred: doohtsʼį́į́ʼ
ref: disínítsʼą́ą́ʼ pred: díítsʼį́į́ʼ
ref: deidiiztsʼą́ą́ʼ pred: dadiitsʼį́į́ʼ
ref: yidiiztsʼą́ą́ʼ pred: yidiitsʼį́į́ʼ
ref: dazdiiztsʼą́ą́ʼ pred: dajíítsʼį́į́ʼ
ref: jidiiztsʼą́ą́ʼ pred: jidiitsʼį́į́ʼ
ref: jidiiztsʼą́ą́ʼ pred: jíítsʼį́į́ʼ
ref: dadidiitsʼį́į́ł pred: dadiitsʼį́į́ł
ref: dideestsʼį́į́ł pred: deeshtsʼį́į́ł
ref: dadidoohtsʼį́į́ł pred: dadoohtsʼį́į́ł
ref: didoohtsʼį́į́ł pred: doohtsʼį́į́ł
ref: deididootsʼį́į́ł pred: dadidootsʼį́į́ł
ref: daz

deiilchʼal deiilchʼal
yíłchʼal yíłchʼal
ref: daoołchʼal pred: dasoołchʼal
woołchʼal woołchʼal
ref: yíníłchʼal pred: shíníłchʼal
yiyííłchʼal yiyííłchʼal
yiyííłchʼal yiyííłchʼal
dajííłchʼal dajííłchʼal
ref: jííłchʼal pred: joołchʼal
jííłchʼal jííłchʼal
deeshchʼał deeshchʼał
ref: deidoołchʼał pred: dadoołchʼał
ref: yidoołchʼał pred: doołchʼał
ref: yidoołchʼał pred: doołchʼał
dazhdoołchʼał dazhdoołchʼał
jidoołchʼał jidoołchʼał
jidoołchʼał jidoołchʼał
deiildin deiildin
yiildin yiildin
yishdin yishdin
daałdin daałdin
wołdin wołdin
niłdin niłdin
deiłdin deiłdin
yiłdin yiłdin
yiłdin yiłdin
dajiłdin dajiłdin
jiłdin jiłdin
jiłdin jiłdin
dasiildin dasiildin
ref: siildin pred: yiildin
ref: séłdin pred: yíłdin
dasoołdin dasoołdin
ref: soołdin pred: wołdin
síníłdin síníłdin
ref: deisdin pred: dayííłdin
ref: yisdin pred: yiłdin
ref: dajisdin pred: dajííłdin
ref: jisdin pred: jííłdin
ref: jisdin pred: jííłdin
deiilghaał deiilghaał
yiilghaał yiilghaał
yishhaał yishhaał
daałhaał daałhaał
wołhaał wołhaał

In [3]:
%cd /dss/dsshome1/03/ge87wod2/morphological-inflection/ByT5/

/dss/dsshome1/03/ge87wod2/morphological-inflection/ByT5


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
